In [1]:
#initialize model
from loss import ContrastiveLoss
import torch
from ml_training import *
from dl_training import *


In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Resnet 50

In [3]:
from models.resnet import ResNet50
criterion=ContrastiveLoss()
res50_model=ResNet50()
opt=torch.optim.Adam(params=res50_model.parameters(),lr=0.0001)
print(count_parameters(res50_model))

24577408


In [4]:
#train model
res50_model_trained=train_dl(50,res50_model,"cuda",criterion,opt)

In [5]:
#extract features
fracture=get_features('../dataset/train/Fracture',res50_model_trained)
normal= get_features('../dataset/train/Normal',res50_model_trained)

In [6]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [7]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [8]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.008386,0.080326,0.022361,0.014591
score_time,0.008395,0.008595,0.005984,0.016351
test_accuracy,0.815152,0.815330,0.791266,0.773262
train_accuracy,0.848018,0.915733,0.875085,0.826942
test_precision_macro,0.839025,0.837306,0.811131,0.795143
train_precision_macro,0.846907,0.914976,0.871521,0.833297
test_recall_macro,0.820113,0.829774,0.791805,0.783534
train_recall_macro,0.853631,0.924282,0.878469,0.839311
test_f1_macro,0.807645,0.808029,0.781803,0.766794
train_f1_macro,0.846534,0.915098,0.873429,0.826490


In [9]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',res50_model_trained)
test_normal= get_features('../dataset/test/Normal',res50_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [10]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [11]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.73      0.73      0.73        15

    accuracy                           0.70        27
   macro avg       0.70      0.70      0.70        27
weighted avg       0.70      0.70      0.70        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.72      0.87      0.79        15

    accuracy                           0.74        

# mobile net

In [12]:
from models.mobilenet import MobileNetV2
criterion=ContrastiveLoss()
mobilenet=MobileNetV2()
opt=torch.optim.Adam(params=mobilenet.parameters(),lr=0.0001)
print(count_parameters(mobilenet))

2879168


In [13]:
#train model
mobilenet_trained=train_dl(50,mobilenet,"cuda",criterion,opt)


In [14]:
#extract features
fracture=get_features('../dataset/train/Fracture',mobilenet_trained)
normal= get_features('../dataset/train/Normal',mobilenet_trained)

In [15]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [16]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [17]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.010976,0.120387,0.018152,0.014925
score_time,0.007978,0.010173,0.007180,0.014960
test_accuracy,0.579144,0.536364,0.530303,0.596613
train_accuracy,1.000000,0.993985,1.000000,0.719891
test_precision_macro,0.584649,0.513479,0.499862,0.587120
train_precision_macro,1.000000,0.994904,1.000000,0.716515
test_recall_macro,0.583459,0.516805,0.505902,0.574549
train_recall_macro,1.000000,0.992857,1.000000,0.704563
test_f1_macro,0.574803,0.505757,0.489797,0.565725
train_f1_macro,1.000000,0.993808,1.000000,0.705733


In [18]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',mobilenet_trained)
test_normal= get_features('../dataset/test/Normal',mobilenet_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [19]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [20]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.36      0.33      0.35        12
           1       0.50      0.53      0.52        15

    accuracy                           0.44        27
   macro avg       0.43      0.43      0.43        27
weighted avg       0.44      0.44      0.44        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.44      0.33      0.38        12
           1       0.56      0.67      0.61        15

    accuracy                           0.52        27
   macro avg       0.50      0.50      0.49        27
weighted avg       0.51      0.52      0.51        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.33      0.25      0.29        12
           1       0.50      0.60      0.55        15

    accuracy                           0.44        

# Squeeze net

In [21]:
from models.squeezenet import SqueezeNetV1
criterion=ContrastiveLoss()
squeezenet=SqueezeNetV1()
opt=torch.optim.Adam(params=squeezenet.parameters(),lr=0.0001)
print(count_parameters(squeezenet))

984000


In [22]:
#train model
squeezenet_trained=train_dl(50,squeezenet,"cuda",criterion,opt)

In [23]:
#extract features
fracture=get_features('../dataset/train/Fracture',squeezenet_trained)
normal= get_features('../dataset/train/Normal',squeezenet_trained)

In [24]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [25]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [26]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.011775,0.119706,0.025732,0.015758
score_time,0.009375,0.009574,0.007881,0.017154
test_accuracy,0.584670,0.572193,0.572193,0.495009
train_accuracy,1.000000,1.000000,1.000000,1.000000
test_precision_macro,0.576965,0.559713,0.560750,0.472833
train_precision_macro,1.000000,1.000000,1.000000,1.000000
test_recall_macro,0.576917,0.554850,0.552970,0.477820
train_recall_macro,1.000000,1.000000,1.000000,1.000000
test_f1_macro,0.575958,0.549063,0.545045,0.472655
train_f1_macro,1.000000,1.000000,1.000000,1.000000


In [27]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',squeezenet_trained)
test_normal= get_features('../dataset/test/Normal',squeezenet_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [28]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [29]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.44      0.67      0.53        12
           1       0.56      0.33      0.42        15

    accuracy                           0.48        27
   macro avg       0.50      0.50      0.47        27
weighted avg       0.51      0.48      0.47        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.50      0.50      0.50        12
           1       0.60      0.60      0.60        15

    accuracy                           0.56        27
   macro avg       0.55      0.55      0.55        27
weighted avg       0.56      0.56      0.56        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.55      0.50      0.52        12
           1       0.62      0.67      0.65        15

    accuracy                           0.59        

# My Model V1

In [30]:
from models.mymodels import SqueezeNetV2
criterion=ContrastiveLoss()
v2_model=SqueezeNetV2()
opt=torch.optim.Adam(params=v2_model.parameters(),lr=0.0001)
print(count_parameters(v2_model))

5781184


In [31]:
#train model
v2_model_trained=train_dl(50,v2_model,"cuda",criterion,opt)

In [32]:
#extract features
fracture=get_features('../dataset/train/Fracture',v2_model_trained)
normal= get_features('../dataset/train/Normal',v2_model_trained)

In [33]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [34]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

,dt,svm,lr,knn
fit_time,0.010976,0.102526,0.096445,0.017153
score_time,0.009177,0.010173,0.008577,0.015360
test_accuracy,0.730481,0.730838,0.718717,0.663636
train_accuracy,0.751584,0.707974,0.759125,1.000000
test_precision_macro,0.758713,0.751985,0.724284,0.665426
train_precision_macro,0.769013,0.700427,0.753347,1.000000
test_recall_macro,0.701353,0.721729,0.712820,0.655526
train_recall_macro,0.723342,0.694762,0.750120,1.000000
test_f1_macro,0.706038,0.721217,0.712008,0.653881
train_f1_macro,0.728242,0.695495,0.751340,1.000000


In [36]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v2_model_trained)
test_normal= get_features('../dataset/test/Normal',v2_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [37]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [38]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.50      0.25      0.33        12
           1       0.57      0.80      0.67        15

    accuracy                           0.56        27
   macro avg       0.54      0.53      0.50        27
weighted avg       0.54      0.56      0.52        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.54      0.58      0.56        12
           1       0.64      0.60      0.62        15

    accuracy                           0.59        27
   macro avg       0.59      0.59      0.59        27
weighted avg       0.60      0.59      0.59        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.64      0.58      0.61        12
           1       0.69      0.73      0.71        15

    accuracy                           0.67        

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# My Model V2

In [39]:
from models.mymodels import SqueezeNetV3
criterion=ContrastiveLoss()
v3_model=SqueezeNetV3()
opt=torch.optim.Adam(params=v3_model.parameters(),lr=0.0001)
print(count_parameters(v3_model))

774768


In [40]:
#train model
v3_model_trained=train_dl(50,v3_model,"cuda",criterion,opt)

In [41]:
#extract features
fracture=get_features('../dataset/train/Fracture',v3_model_trained)
normal= get_features('../dataset/train/Normal',v3_model_trained)

In [42]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [43]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [44]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.010578,0.103027,0.029925,0.020744
score_time,0.010572,0.011569,0.010372,0.022540
test_accuracy,0.808378,0.808378,0.790374,0.790374
train_accuracy,0.864548,0.825382,0.820893,0.817840
test_precision_macro,0.846762,0.849252,0.845073,0.825632
train_precision_macro,0.863434,0.823102,0.816566,0.817563
test_recall_macro,0.810564,0.808684,0.787519,0.798797
train_recall_macro,0.867968,0.826846,0.817105,0.824180
test_f1_macro,0.800518,0.799077,0.777145,0.782253
train_f1_macro,0.862874,0.822861,0.816705,0.816363


In [45]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v3_model_trained)
test_normal= get_features('../dataset/test/Normal',v3_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [46]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [47]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.70      0.58      0.64        12
           1       0.71      0.80      0.75        15

    accuracy                           0.70        27
   macro avg       0.70      0.69      0.69        27
weighted avg       0.70      0.70      0.70        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.82      0.75      0.78        12
           1       0.81      0.87      0.84        15

    accuracy                           0.81        27
   macro avg       0.82      0.81      0.81        27
weighted avg       0.82      0.81      0.81        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.72      0.87      0.79        15

    accuracy                           0.74        

# My Model V3

In [48]:
from models.mymodels import SqueezeNetV4
criterion=ContrastiveLoss()
v4_model=SqueezeNetV4()
opt=torch.optim.Adam(params=v4_model.parameters(),lr=0.0001)
print(count_parameters(v4_model))

774768


In [49]:
#train model
v4_model_trained=train_dl(50,v4_model,"cuda",criterion,opt)

In [50]:
#extract features
fracture=get_features('../dataset/train/Fracture',v4_model_trained)
normal= get_features('../dataset/train/Normal',v4_model_trained)

In [51]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [52]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [53]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.012566,0.095943,0.095545,0.026729
score_time,0.015558,0.014561,0.012966,0.024734
test_accuracy,0.808556,0.772193,0.754011,0.778431
train_accuracy,0.847961,0.804261,0.835897,1.000000
test_precision_macro,0.831769,0.792048,0.771083,0.801346
train_precision_macro,0.856308,0.799880,0.832761,1.000000
test_recall_macro,0.819962,0.772820,0.745752,0.782105
train_recall_macro,0.862312,0.803691,0.829571,1.000000
test_f1_macro,0.803355,0.766365,0.743755,0.772420
train_f1_macro,0.847614,0.800876,0.830954,1.000000


In [54]:
#embeddings of test data
test_fracture=get_features('../dataset/test/Fracture',v4_model_trained)
test_normal= get_features('../dataset/test/Normal',v4_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [55]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [56]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       1.00      0.75      0.86        12
           1       0.83      1.00      0.91        15

    accuracy                           0.89        27
   macro avg       0.92      0.88      0.88        27
weighted avg       0.91      0.89      0.89        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       1.00      0.42      0.59        12
           1       0.68      1.00      0.81        15

    accuracy                           0.74        